In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [57]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D,GRU
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
import numpy as np
import pandas as pd

In [3]:
EMBEDDING_FILE = "/mnt/c/Users/Anirudh/Desktop/glove.840B.300d.txt"
train = pd.read_csv("/mnt/c/Users/Anirudh/Desktop/train.tsv",delimiter="\t").fillna(" ")
test = pd.read_csv("/mnt/c/Users/Anirudh/Desktop/test_data.tsv",delimiter="\t").fillna(" ")

In [4]:
max_features = 100000
maxlen = 150
embed_size=300

In [5]:
train.head()

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate


In [6]:
train["Label"].value_counts()

moderate          6019
not depression    1971
severe             901
Name: Label, dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train["Label"] = le.fit_transform(train["Label"])

In [8]:
train["Label"].value_counts()

0    6019
1    1971
2     901
Name: Label, dtype: int64

In [9]:
train.head()

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,0
1,train_pid_2,My new years resolution : I'm gonna get my ass...,0
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,0
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",0
4,train_pid_5,Sat in the dark and cried myself going into th...,0


In [10]:
df=train[0:901]
df2=train[6019:6920]
df3=train[7990:]

df = df[['Text_data', 'Label']]
df2 = df2[['Text_data', 'Label']]
df3 = df3[['Text_data', 'Label']]

print("Moderate cases: ", str(len(df)))
print("Not Depressed cases: ", str(len(df2)))
print("Severe cases: ", str(len(df3)))

data = pd.concat([df, df2, df3], axis=0).reset_index(drop=True)

Moderate cases:  901
Not Depressed cases:  901
Severe cases:  901


In [11]:
train_x = data["Text_data"].str.lower()
train_y = data["Label"]
train_y = pd.get_dummies(train_y,columns=train_y).values
train_y.shape

(2703, 3)

In [12]:
test_x =test["text data"].str.lower()

In [13]:
tokenizer = text.Tokenizer(num_words=max_features,lower=True)
tokenizer.fit_on_texts(list(train_x))

In [14]:
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [15]:
train_x = sequence.pad_sequences(train_x,maxlen=maxlen)
test_x = sequence.pad_sequences(test_x,maxlen=maxlen)

In [16]:
embedding_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.strip().rsplit(" ")
        word = values[0]
        coefs = np.asarray(values[1:],dtype='float32')
        embedding_index[word] = coefs

In [17]:
word_index = tokenizer.word_index
num_words = min(len(word_index)+1,max_features)
embedding_matrix = np.zeros((num_words,embed_size))

for word, i in word_index.items():
    if i>=max_features:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# basic N network

In [18]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.35)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

2022-03-02 18:59:34.085352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-02 18:59:34.215228: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-02 18:59:34.215861: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-02 18:59:34.217751: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [19]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5


2022-03-02 18:59:45.664562: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-03-02 18:59:48.522776: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied


85/85 [==============================] - 16s 44ms/step - loss: 0.8419 - accuracy: 0.5942
Epoch 2/5
85/85 [==============================] - 3s 41ms/step - loss: 0.4646 - accuracy: 0.8302
Epoch 3/5
85/85 [==============================] - 3s 41ms/step - loss: 0.2847 - accuracy: 0.9042
Epoch 4/5
85/85 [==============================] - 3s 41ms/step - loss: 0.1710 - accuracy: 0.9486
Epoch 5/5
85/85 [==============================] - 4s 42ms/step - loss: 0.1216 - accuracy: 0.9656


In [20]:
new_complaint =test['text data']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = sequence.pad_sequences(seq, maxlen=maxlen, truncating='post', padding='post')
pred = model.predict(padded)
labels = ['moderate','not depression','severe']

In [21]:
test

,Pid,text data
0,test_pid_1,Im scared : This is it. I lie to myself every ...
1,test_pid_2,New to this but just wanted to vent : I just f...
2,test_pid_3,I’m sad : It’s kinda always been an issue. I w...
3,test_pid_4,Lonely but not alone. : All of my immediately ...
4,test_pid_5,This year has been trash. : I dont know why I’...
...,...,...
3240,test_pid_3241,"Feeling lonely. : Hi reddit, I haven’t posted ..."
3241,test_pid_3242,When would suicide be right? : So I got back f...
3242,test_pid_3243,Lowest I’ve ever been ever. : To make a long s...
3243,test_pid_3244,Does the Toxoplasma Gondii ruined my life ? (f...


In [22]:
file=[]
for i in range(0,3245):
    file.append(labels[np.argmax(pred[i])])

In [40]:
new_complaint =['not feeling happy']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = sequence.pad_sequences(seq, maxlen=maxlen, truncating='post', padding='post')
pred = model.predict(padded)
labels = ['moderate','not depression','severe']
print(pred, labels[np.argmax(pred)])

[[0.8870026  0.10421845 0.00877903]] moderate


# conv layer tuning 

2 conv layers and changing number of neurons


In [41]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.35)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)
x = Conv1D(128,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [42]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 12s 43ms/step - loss: 0.8398 - accuracy: 0.5990
Epoch 2/5
85/85 [==============================] - 4s 43ms/step - loss: 0.4777 - accuracy: 0.8217
Epoch 3/5
85/85 [==============================] - 4s 41ms/step - loss: 0.2852 - accuracy: 0.9027
Epoch 4/5
85/85 [==============================] - 4s 43ms/step - loss: 0.1717 - accuracy: 0.9412
Epoch 5/5
85/85 [==============================] - 4s 43ms/step - loss: 0.1361 - accuracy: 0.9575


3 conv layers

In [43]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.35)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)
x = Conv1D(128,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)
x = Conv1D(256,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [44]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 7s 47ms/step - loss: 0.8300 - accuracy: 0.6053
Epoch 2/5
85/85 [==============================] - 4s 46ms/step - loss: 0.4832 - accuracy: 0.8098
Epoch 3/5
85/85 [==============================] - 4s 46ms/step - loss: 0.2882 - accuracy: 0.9042
Epoch 4/5
85/85 [==============================] - 4s 44ms/step - loss: 0.1912 - accuracy: 0.9345
Epoch 5/5
85/85 [==============================] - 4s 45ms/step - loss: 0.1574 - accuracy: 0.9501


# Dropout hypertuning

In [49]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.1))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [50]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 6s 42ms/step - loss: 0.7864 - accuracy: 0.6374
Epoch 2/5
85/85 [==============================] - 4s 43ms/step - loss: 0.4197 - accuracy: 0.8616
Epoch 3/5
85/85 [==============================] - 4s 43ms/step - loss: 0.2215 - accuracy: 0.9282
Epoch 4/5
85/85 [==============================] - 4s 43ms/step - loss: 0.1149 - accuracy: 0.9689
Epoch 5/5
85/85 [==============================] - 4s 43ms/step - loss: 0.0735 - accuracy: 0.9815


In [51]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.1)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.05))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [52]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 6s 42ms/step - loss: 0.7447 - accuracy: 0.6574
Epoch 2/5
85/85 [==============================] - 4s 42ms/step - loss: 0.3388 - accuracy: 0.8868
Epoch 3/5
85/85 [==============================] - 4s 42ms/step - loss: 0.1754 - accuracy: 0.9493
Epoch 4/5
85/85 [==============================] - 4s 42ms/step - loss: 0.0878 - accuracy: 0.9771
Epoch 5/5
85/85 [==============================] - 4s 42ms/step - loss: 0.0484 - accuracy: 0.9889


In [53]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.5)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.2))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [54]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 6s 42ms/step - loss: 0.9272 - accuracy: 0.5438
Epoch 2/5
85/85 [==============================] - 4s 42ms/step - loss: 0.6203 - accuracy: 0.7543
Epoch 3/5
85/85 [==============================] - 3s 41ms/step - loss: 0.4003 - accuracy: 0.8557
Epoch 4/5
85/85 [==============================] - 4s 43ms/step - loss: 0.2819 - accuracy: 0.9038
Epoch 5/5
85/85 [==============================] - 4s 42ms/step - loss: 0.1977 - accuracy: 0.9327


Layer hypertuning

In [55]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.1)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [56]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 2s 13ms/step - loss: 0.6887 - accuracy: 0.7222
Epoch 2/5
85/85 [==============================] - 1s 12ms/step - loss: 0.2958 - accuracy: 0.9382
Epoch 3/5
85/85 [==============================] - 1s 12ms/step - loss: 0.1604 - accuracy: 0.9674
Epoch 4/5
85/85 [==============================] - 1s 12ms/step - loss: 0.0884 - accuracy: 0.9863
Epoch 5/5
85/85 [==============================] - 1s 12ms/step - loss: 0.0550 - accuracy: 0.9930


In [58]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.5)(x)
x = GRU(128,return_sequences=True,dropout=0.2)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [59]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 4s 24ms/step - loss: 0.9398 - accuracy: 0.5376
Epoch 2/5
85/85 [==============================] - 2s 22ms/step - loss: 0.6624 - accuracy: 0.7266
Epoch 3/5
85/85 [==============================] - 2s 24ms/step - loss: 0.4572 - accuracy: 0.8383
Epoch 4/5
85/85 [==============================] - 2s 23ms/step - loss: 0.3379 - accuracy: 0.8849
Epoch 5/5
85/85 [==============================] - 2s 23ms/step - loss: 0.2459 - accuracy: 0.9179


In [60]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.1)(x)
x = LSTM(128,return_sequences=True,dropout=0.05)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [61]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 4s 26ms/step - loss: 0.8059 - accuracy: 0.6360
Epoch 2/5
85/85 [==============================] - 2s 26ms/step - loss: 0.4459 - accuracy: 0.8420
Epoch 3/5
85/85 [==============================] - 2s 26ms/step - loss: 0.2250 - accuracy: 0.9353
Epoch 4/5
85/85 [==============================] - 2s 26ms/step - loss: 0.1234 - accuracy: 0.9652
Epoch 5/5
85/85 [==============================] - 2s 25ms/step - loss: 0.0557 - accuracy: 0.9859


# Final resultant model

In [62]:
inp = Input(shape=(maxlen,))
x = Embedding(num_words,embed_size,weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.1)(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.05))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [63]:
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=5, verbose=1)

Epoch 1/5
85/85 [==============================] - 7s 42ms/step - loss: 0.7661 - accuracy: 0.6611
Epoch 2/5
85/85 [==============================] - 3s 41ms/step - loss: 0.3413 - accuracy: 0.8879
Epoch 3/5
85/85 [==============================] - 3s 41ms/step - loss: 0.1528 - accuracy: 0.9534
Epoch 4/5
85/85 [==============================] - 3s 40ms/step - loss: 0.0886 - accuracy: 0.9771
Epoch 5/5
85/85 [==============================] - 4s 43ms/step - loss: 0.0471 - accuracy: 0.9878


In [67]:
new_complaint =['want to die']  #trigger word die
seq = tokenizer.texts_to_sequences(new_complaint)
padded = sequence.pad_sequences(seq, maxlen=maxlen, truncating='post', padding='post')
pred = model.predict(padded)
labels = ['moderate','not depression','severe']
print(pred, labels[np.argmax(pred)])

[[0.31516716 0.18065454 0.5041783 ]] severe
